In [8]:
import math
from collections import defaultdict
from statistics import NormalDist

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from prosperity3.analysis.data import read_price_data

cdf = NormalDist().cdf

In [2]:
df = read_price_data([(3, 2)], lambda row: row.mid_price)

days_to_expiry = 6
strike_prices = [9500, 9750, 10000, 10250, 10500]

df

,PICNIC_BASKET2,VOLCANIC_ROCK_VOUCHER_9750,RAINFOREST_RESIN,VOLCANIC_ROCK_VOUCHER_9500,VOLCANIC_ROCK,SQUID_INK,VOLCANIC_ROCK_VOUCHER_10250,KELP,DJEMBES,CROISSANTS,JAMS,VOLCANIC_ROCK_VOUCHER_10000,VOLCANIC_ROCK_VOUCHER_10500,PICNIC_BASKET1
0,30096.5,469.5,10000.0,718.5,10218.5,1800.5,63.5,2045.5,13419.5,4265.5,6519.5,233.5,8.5,58707.0
1,30097.5,469.0,9995.5,718.0,10217.5,1801.0,62.5,2045.0,13419.5,4265.5,6520.0,234.5,8.5,58712.0
2,30094.5,473.5,10000.0,722.5,10222.0,1803.5,64.5,2046.5,13419.5,4265.5,6519.5,235.5,9.5,58715.5
3,30095.5,474.5,10000.0,723.5,10223.0,1802.5,65.5,2045.5,13419.5,4265.5,6519.5,235.5,9.5,58716.5
4,30099.5,468.5,10000.0,717.5,10217.0,1801.5,62.5,2045.5,13419.5,4266.5,6519.5,234.5,8.5,58715.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30073.0,418.5,10000.0,668.0,10168.0,1823.0,36.5,2041.0,13296.5,4239.5,6526.0,182.5,3.5,58410.5
9996,30069.0,418.5,9997.5,667.5,10167.0,1823.0,36.5,2041.0,13295.5,4239.5,6526.0,184.5,3.5,58415.5
9997,30069.0,418.5,10000.0,667.5,10167.0,1823.5,36.5,2041.5,13295.5,4239.5,6526.0,182.5,3.5,58415.0
9998,30072.0,418.5,10000.0,667.5,10167.5,1822.5,36.5,2041.5,13296.5,4240.5,6526.0,185.5,3.5,58420.5


In [3]:
def black_scholes(
    asset_price: float, strike_price: float, expiration_time: float, risk_free_rate: float, volatility: float
) -> float:
    # fmt: off
    d1 = (math.log(asset_price / strike_price) + (risk_free_rate + volatility**2 / 2) * expiration_time) / (volatility * math.sqrt(expiration_time))
    d2 = d1 - volatility * math.sqrt(expiration_time)
    return asset_price * cdf(d1) - strike_price * math.exp(-risk_free_rate * expiration_time) * cdf(d2)
    # fmt: on

In [4]:
def implied_volatility(asset_price: float, strike_price: float, voucher_price: float, expiration_time: float) -> float:
    risk_free_rate = 0

    lo, hi = 0.01, 1.0
    volatility = (lo + hi) / 2

    max_iterations = 200
    tolerance = 1e-6

    for _ in range(max_iterations):
        expected_price = black_scholes(asset_price, strike_price, expiration_time, risk_free_rate, volatility)
        delta = expected_price - voucher_price

        if abs(delta) < tolerance:
            break

        if delta > 0:
            hi = volatility
        else:
            lo = volatility

        volatility = (lo + hi) / 2

    return volatility


implied_volatility(df["VOLCANIC_ROCK"].iloc[0], 9500, df["VOLCANIC_ROCK_VOUCHER_9500"].iloc[0], days_to_expiry / 365)

0.07187500000000001

In [5]:
timestamps = [0, 1000, 2500, 5000]
fig = make_subplots(rows=len(timestamps), cols=1, subplot_titles=[f"t={v:,.0f}" for v in timestamps])

for i, t in enumerate(timestamps):
    m_t = []
    v_t = []

    expiration_time = days_to_expiry / 365 - (t / 10_000 / 365)

    asset_price = df["VOLCANIC_ROCK"].iloc[t]
    for strike_price in strike_prices:
        voucher_price = df[f"VOLCANIC_ROCK_VOUCHER_{strike_price}"].iloc[t]

        m = np.log(strike_price / asset_price) / np.sqrt(expiration_time)
        iv = implied_volatility(asset_price, strike_price, voucher_price, expiration_time)

        m_t.append(m)
        v_t.append(iv)

    v_fit = np.poly1d(np.polyfit(m_t, v_t, deg=2))

    base_iv = v_fit(0)
    print(f"t={t} base IV: {base_iv}")

    for strike_price in strike_prices:
        voucher_price = df[f"VOLCANIC_ROCK_VOUCHER_{strike_price}"].iloc[t]

        m = np.log(strike_price / asset_price) / np.sqrt(expiration_time)
        iv = implied_volatility(asset_price, strike_price, voucher_price, expiration_time)
        iv_expected = v_fit(m)

        print(f"t={t} strike={strike_price} iv={iv} iv_expected={iv_expected} diff={iv - iv_expected}")

    m_range = np.linspace(min(m_t) - 2, max(m_t) + 2, 200)
    v_range = [v_fit(m) for m in m_range]

    fig.add_trace(
        go.Scatter(x=m_t, y=v_t, name="Observed IVs", mode="markers", marker={"color": "red"}), row=i + 1, col=1
    )
    fig.add_trace(go.Scatter(x=m_range, y=v_range, name="Fitted Smile", line={"color": "green"}), row=i + 1, col=1)

fig.update_layout(height=len(strike_prices) * 200)
fig.update_xaxes(title_text="Moneyness")
fig.update_yaxes(title_text="Implied Volatility")
fig.show()

t=0 base IV: 0.16870542969967917
t=0 strike=9500 iv=0.07187500000000001 iv_expected=0.08389705113920902 diff=-0.012022051139209011
t=0 strike=9750 iv=0.1654824638366699 iv_expected=0.1383497070702709 diff=0.027132756766399008
t=0 strike=10000 iv=0.15802524484694003 iv_expected=0.16560498858926279 diff=-0.007579743742322759
t=0 strike=10250 iv=0.149471070766449 iv_expected=0.16763591141222156 diff=-0.018164840645772562
t=0 strike=10500 iv=0.15687672361731528 iv_expected=0.14624284485640995 diff=0.010633878760905324
t=1000 base IV: 0.14953387444312355
t=1000 strike=9500 iv=0.20888033747673038 iv_expected=0.2081327852738088 diff=0.0007475522029215809
t=1000 strike=9750 iv=0.17052349463105201 iv_expected=0.17254438225794952 diff=-0.002020887626897505
t=1000 strike=10000 iv=0.15503162801265719 iv_expected=0.15353367328555537 diff=0.001497954727101819
t=1000 strike=10250 iv=0.14997674126178026 iv_expected=0.1498995210927583 diff=7.722016902195206e-05
t=1000 strike=10500 iv=0.1602440050244331

In [6]:
diffs = defaultdict(list)

for t in range(df.shape[0]):
    asset_price = df["VOLCANIC_ROCK"].iloc[t]
    expiration_time = days_to_expiry / 365 - (t / 10_000 / 365)

    ivs = {}
    for strike_price in strike_prices:
        voucher_price = df[f"VOLCANIC_ROCK_VOUCHER_{strike_price}"].iloc[t]
        ivs[strike_price] = implied_volatility(asset_price, strike_price, voucher_price, expiration_time)

    m_t = []
    v_t = []
    for strike_price in strike_prices:
        m_t.append(np.log(strike_price / asset_price) / np.sqrt(expiration_time))
        v_t.append(ivs[strike_price])

    v_fit = np.poly1d(np.polyfit(m_t, v_t, deg=2))

    for i, strike_price in enumerate(strike_prices):
        diffs[strike_price].append(v_t[i] - v_fit(m_t[i]))

fig = make_subplots(rows=1, cols=1)

for i, strike_price in enumerate(strike_prices):
    x = np.arange(0, df.shape[0] * 100, 100)
    y = diffs[strike_price]
    fig.add_trace(go.Scatter(x=x, y=y, name=f"{strike_price:,.0f}"))

fig.update_layout(height=500)
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Diff")
fig.show()

In [20]:
ivs = defaultdict(list)

for t in range(df.shape[0]):
    asset_price = df["VOLCANIC_ROCK"].iloc[t]
    expiration_time = days_to_expiry / 365 - (t / 10_000 / 365)

    for strike_price in strike_prices:
        voucher_price = df[f"VOLCANIC_ROCK_VOUCHER_{strike_price}"].iloc[t]
        ivs[strike_price].append(implied_volatility(asset_price, strike_price, voucher_price, expiration_time))

fig = make_subplots(rows=2, cols=1)

for i, strike_price in enumerate(strike_prices):
    x = np.arange(0, df.shape[0] * 100, 100)
    y_prices = df[f"VOLCANIC_ROCK_VOUCHER_{strike_price}"]

    y_signal = pd.Series(ivs[strike_price])
    y_signal = ((y_signal - y_signal.rolling(100).mean()) / y_signal.rolling(100).std()).rolling(10).mean()

    fig.add_trace(go.Scatter(x=x, y=y_prices, name=f"Price: {strike_price:,.0f}"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=y_signal, name=f"Signal: {strike_price:,.0f}"), row=2, col=1)

fig.update_layout(height=800)
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="IV")
fig.show()